# Lab 7.1: Most Popular Superhero

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu. Bạn sẽ cần viết một chương trình để dựa theo tập dữ liệu để đưa ra siêu tên của siêu anh hùng có số lượng kết nối nhiều nhất, và tính được xem trung bình một siêu anh hùng có bao nhiêu kết nối.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/drive/folders/1_f81WTm1fCHG4HMUWM7JOnqVvqfofsSs?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu gồm 2 file như sau:

1. `Marvel_Names.csv`

File .csv chứa thông tin về các siêu anh hùng với cấu trúc như sau:
- `Id`: Id của siêu anh hùng
- `Name`: Tên của siêu anh hùng đó.

2. `Marvel_Graph.txt`

File .txt chứa các thông tin về kết nối giữa các siêu anh hùng, sẽ có dạng như sau: `1602 3889 1535 6038 533 3986`

Dữ liệu này nghĩa là siêu anh hùng với ID là `1602` có kết nối đến các siêu anh hùng `3889 1535 6038 533 3986`.


Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Find the Most Popular Superhero in a Social Graph](https://funix.udemy.com/course/taming-big-data-with-apache-spark-hands-on/learn/lecture/3726116#questions)


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

config = SparkConf().setMaster('local').setAppName('lab53')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

DATASET_PATH_NAME = '/content/gdrive/MyDrive/Marvel_Names.csv'
DATASET_PATH_GRAPH = '/content/gdrive/MyDrive/Marvel_Graph.txt'

In [8]:
schema = StructType([
    StructField("Id", IntegerType()),
    StructField("Name", StringType()),
	])

names = spark.read \
		.schema(schema) \
		.csv(DATASET_PATH_NAME)
names.show()

lines = spark.read.text(DATASET_PATH_GRAPH)
lines.show()

+---+--------------------+
| Id|                Name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
|  6|               A'YIN|
|  7|        ABBOTT, JACK|
|  8|             ABCISSA|
|  9|                ABEL|
| 10|ABOMINATION/EMIL BLO|
| 11|ABOMINATION | MUTANT|
| 12|         ABOMINATRIX|
| 13|             ABRAXAS|
| 14|          ADAM 3,031|
| 15|             ABSALOM|
| 16|ABSORBING MAN/CARL C|
| 17|ABSORBING MAN | MUTA|
| 18|                ACBA|
| 19|ACHEBE, REVEREND DOC|
| 20|            ACHILLES|
+---+--------------------+
only showing top 20 rows

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
|5981 3569 5353 40...|
|5986 2658 3712 26...|
|5987 2614 5716 17...|
|5984 590 4898 745...|
|5985 3233 2254 21...|
|6294 4898 1127 32...|
|270

Tạo một Dataframe để tính số lượng kết nối của một siêu anh hùng, ví dụ:
```
+----+-----------+
|  id|connections|
+----+-----------+
| 691|          6|
|1159|         11|
|3959|        142|
|1572|         35|
+----+-----------+
```

Lưu ý rằng một id của siêu anh hùng có thể xuất hiện nhiều lần nên bạn sẽ cần tính tổng, để trường id không có các dữ liệu trùng lặp.

In [ ]:
from pyspark.sql import functions as f
connections = lines.withColumn("id", f.split(f.col('value'), " ")[0]) \
                    .withColumn('CountConnect', f.size(f.split(f.col('value'), " ")) - 1)\
                    .groupBy('id').agg(f.sum('CountConnect').alias('CountConnect'))
connections.show()

In [13]:
# Tính số kết nối trung bình của một siêu anh hùng.
avg_df = connections.select(f.avg('CountConnect'))

avg_df.show()

+-----------------+
|avg(CountConnect)|
+-----------------+
|52.90209682392846|
+-----------------+



In [ ]:
# Join 2 Dataframe để lấy được dữ liệu về tên của siêu anh hùng
join_expr = connections.id == names.Id
joined_df = connections \
              .join(names, join_expr, 'inner') \
              .drop(names.Id)

joined_df.show()

In [24]:
# Tìm tên siêu anh hùng có nhiều kết nối nhất
mostPopular = connections.sort(f.col('CountConnect').desc()).first()
name_popular = names.filter(f.col('Id') == mostPopular[0]).select('Name').first()
print(name_popular[0] 	+ " is the most popular superhero with " 	+ str(mostPopular[1]) 	+ " co-appearances." )

CAPTAIN AMERICA is the most popular superhero with 1937 co-appearances.
